In [1]:
import os
os.environ["SCIPY_ARRAY_API"] = "1"

In [2]:
import pandas as pd
import warnings
from inspect import signature
from sklearn.utils import all_estimators
from sklearn.exceptions import ConvergenceWarning
import threadpoolctl

from sample_weight_audit import check_weighted_repeated_estimator_fit_equivalence
from sample_weight_audit.sklearn_stochastic_params import STOCHASTIC_FIT_PARAMS 


threadpoolctl.threadpool_limits(limits=1, user_api="openmp")  # HistGradientBoostingClassifier is trashing.
warnings.filterwarnings("ignore", category=RuntimeWarning)  # division by zero in AdaBoost
warnings.filterwarnings("ignore", category=ConvergenceWarning)  # Liblinear can fail to converge

In [ ]:
results = []
# XXX: how to handle clustering estimators?
for est_name, est_class in all_estimators(
    type_filter=["classifier", "regressor", "transformer"]
):
    if "random_state" not in signature(est_class.__init__).parameters:
        # Skip estimators with a deterministic fit: they are better tested via
        # the check_sample_weight_equivalence_on_dense_data estimator check run
        # as part of the scikit-learn test suite.
        continue

    if "sample_weight" not in signature(est_class.fit).parameters:
        print(f"❌ {est_name} does not support sample_weight")
        continue

    print(f"Evaluating {est_name}")
    est = est_class(**STOCHASTIC_FIT_PARAMS.get(est_class, {}))
    try:
        result = check_weighted_repeated_estimator_fit_equivalence(
            est,
            test_name="kstest",
            n_stochastic_fits=30,
        )
        pass_or_fail = "✅" if result.min_p_value > 0.05 else "❌"
        print(
            f"{pass_or_fail} {est_name}: (min_p_value: {result.min_p_value:.3f}, "
            f"mean_p_value={result.mean_p_value:.3f})"
        )
        results.append(result)
    except ValueError:
        # XXX: Use the deterministic check instead?
        print(f"❌ {est_name} with different random states led to the same predictions")


results_df = pd.DataFrame([r.to_dict() for r in results])

Evaluating AdaBoostClassifier


100%|██████████| 30/30 [00:02<00:00, 10.44it/s]


❌ AdaBoostClassifier with different random states led to the same predictions
Evaluating AdaBoostRegressor


100%|██████████| 30/30 [00:02<00:00, 11.29it/s]


✅ AdaBoostRegressor: (min_p_value: 0.135, mean_p_value=0.724)
Evaluating BaggingClassifier


...

100%|██████████| 30/30 [00:00<00:00, 333.55it/s]


❌ LinearSVR: (min_p_value: 0.000, mean_p_value=0.011)
❌ LocallyLinearEmbedding does not support sample_weight
Evaluating LogisticRegression


100%|██████████| 30/30 [00:00<00:00, 73.00it/s]


❌ LogisticRegression: (min_p_value: 0.000, mean_p_value=0.113)
Evaluating LogisticRegressionCV


 60%|██████    | 18/30 [02:17<01:32,  7.69s/it]

In [ ]:
results_df.sort_values("min_p_value")